In [1]:
import pandas as pd

In [8]:
x = pd.read_csv("meteo_calidad_2015.csv")

In [11]:
print(x)

     Dia  Dia_mes  Mes Dia_sem  Lluvia  T_MAX  T_MIN  Viento_MAX  Viento_MED  \
0      1        1  ENE       J     0.0   10.7   -0.6          14           5   
1      2        2  ENE       V     0.0   12.5   -0.4          11           5   
2      3        3  ENE       S     0.0   11.7    1.4          14           8   
3      4        4  ENE       D     0.0   14.4    1.4          12           6   
4      5        5  ENE       L     0.0   12.3    2.0          19           9   
..   ...      ...  ...     ...     ...    ...    ...         ...         ...   
360  361       27  DIC       D     0.0   13.8    5.9          23          13   
361  362       28  DIC       L     0.7   14.1    6.4          30          14   
362  363       29  DIC       M     0.0   10.7    6.0          22           9   
363  364       30  DIC       X     0.0   10.4    7.0          18           9   
364  365       31  DIC       J     0.6   12.2    9.2          22          10   

      SO2  ...  NO_MAX  NO2_MAX  PM2.5_

In [15]:
print(pd.isnull(x).sum())

Dia           0
Dia_mes       0
Mes           0
Dia_sem       0
Lluvia        0
T_MAX         0
T_MIN         0
Viento_MAX    0
Viento_MED    0
SO2           0
CO            0
NO            0
NO2           0
PM2.5         0
PM10          0
O3            0
TOL           0
BEN           0
EBE           0
TCH           0
NMCH          0
SO2_MAX       0
CO_MAX        0
NO_MAX        0
NO2_MAX       0
PM2.5_MAX     0
PM10_MAX      0
O3_MAX        0
TOL_MAX       0
BEN_MAX       0
EBE_MAX       0
TCH_MAX       0
NMCH_MAX      0
dtype: int64


In [16]:
x.corr()

,Dia,Dia_mes,Lluvia,T_MAX,T_MIN,Viento_MAX,Viento_MED,SO2,CO,NO,...,NO_MAX,NO2_MAX,PM2.5_MAX,PM10_MAX,O3_MAX,TOL_MAX,BEN_MAX,EBE_MAX,TCH_MAX,NMCH_MAX
Dia,1.000000,0.095131,-0.025115,0.190629,0.300988,-0.271643,-0.286813,0.190222,0.076452,0.171726,...,0.198861,0.076015,0.336121,0.254277,-0.221987,0.029124,0.122400,0.021628,-0.443385,-0.739330
Dia_mes,0.095131,1.000000,-0.000395,-0.012283,-0.014743,0.125960,0.054054,-0.064034,-0.098272,-0.108103,...,-0.093850,-0.104356,-0.119935,-0.159312,0.021452,-0.105318,-0.083204,-0.094678,-0.109306,-0.019789
Lluvia,-0.025115,-0.000395,1.000000,-0.120502,-0.022188,0.160457,0.136657,-0.091098,-0.104398,-0.120208,...,-0.119712,-0.130772,-0.110517,-0.148561,0.027092,-0.092900,-0.058365,-0.074858,-0.098632,0.031177
T_MAX,0.190629,-0.012283,-0.120502,1.000000,0.941484,0.149109,0.152920,-0.518337,-0.516272,-0.413155,...,-0.431650,-0.294018,-0.074646,0.279006,0.611183,-0.328394,-0.422632,-0.360226,-0.161037,-0.275766
T_MIN,0.300988,-0.014743,-0.022188,0.941484,1.000000,0.185992,0.183890,-0.573904,-0.596274,-0.490523,...,-0.502518,-0.399331,-0.110633,0.225517,0.588075,-0.354989,-0.449193,-0.404112,-0.243916,-0.347277
Viento_MAX,-0.271643,0.125960,0.160457,0.149109,0.185992,1.000000,0.919605,-0.542835,-0.601700,-0.593030,...,-0.592765,-0.603150,-0.541741,-0.298431,0.565617,-0.457100,-0.567464,-0.490170,-0.236809,0.148022
Viento_MED,-0.286813,0.054054,0.136657,0.152920,0.183890,0.919605,1.000000,-0.521946,-0.577966,-0.573885,...,-0.568227,-0.561899,-0.509416,-0.262519,0.561531,-0.433468,-0.557729,-0.466444,-0.245110,0.140612
SO2,0.190222,-0.064034,-0.091098,-0.518337,-0.573904,-0.542835,-0.521946,1.000000,0.934681,0.912839,...,0.928388,0.844065,0.666969,0.362112,-0.731610,0.609649,0.750767,0.632783,0.283942,-0.035790
CO,0.076452,-0.098272,-0.104398,-0.516272,-0.596274,-0.601700,-0.577966,0.934681,1.000000,0.969395,...,0.964424,0.913700,0.718439,0.408921,-0.761076,0.730351,0.853201,0.745915,0.410836,0.085464
NO,0.171726,-0.108103,-0.120208,-0.413155,-0.490523,-0.593030,-0.573885,0.912839,0.969395,1.000000,...,0.979737,0.892168,0.725074,0.444589,-0.746314,0.740656,0.820628,0.759730,0.386765,-0.011048
